In [ ]:
from net.util import *
import os
import hypertools as hyp
import pickle
import json
from sklearn.preprocessing import Normalizer
import numpy as np
import faiss
from  tqdm import tqdm

def aa(*args, **kwargs):
    group.add_argument(*args, **kwargs)
parser = argparse.ArgumentParser()

group = parser.add_argument_group('dataset options')
aa("--database", default="deep1b") # can be "bigann", "deep1b" or "*.fvecs"
aa("--size_base", type=int, default=int(1e6),
    help="size of evaluation dataset")
aa("--num_learn", type=int, default=int(5e5),
    help="nb of learning vectors")

group = parser.add_argument_group('Model hyperparameters')
aa("--dint", type=int, default=1024,
    help="size of hidden states")
aa("--dout", type=int, default=16,
    help="output dimension")
aa("--lambda_uniform", type=float, default=0.05,
    help="weight of the uniformity loss")
aa("--lambda_seperation", type=float, default=0.05,
    help="weight of the seperation loss")
aa("--lambda_lid", type=float, default=0.01,
    help="weight of the seperation loss")
aa("--lambda_triplet", type=float, default=1,
    help="weight of the seperation loss")

    
group = parser.add_argument_group('Training hyperparameters')
aa("--batch_size", type=int, default=64)
aa("--epochs", type=int, default=160)
aa("--momentum", type=float, default=0.9)
aa("--rank_positive", type=int, default=10,
    help="this number of vectors are considered positives")
aa("--rank_negative", type=int, default=50,
    help="these are considered negatives")

group = parser.add_argument_group('Computation params')
aa("--seed", type=int, default=1234)
aa("--checkpoint_dir", type=str, default="",
    help="checkpoint directory")
aa("--init_name", type=str, default="",
    help="checkpoint to load from")
aa("--save_best_criterion", type=str, default="",
    help="for example r2=4,rank=10")
aa("--quantizer_train", type=str, default="")
aa("--lr_schedule", type=str, default="0.1,0.05,0.01")
aa("--device", choices=["cuda", "cpu", "auto"], default="auto")
aa("--val_freq", type=int, default=10,
    help="frequency of validation calls")
aa("--validation_quantizers", type=str, default="",
    help="r2 values to try in validation")
aa("--update_freq",type=int,default=10,help="update frequence")

aa("--margin_k",type=int,default=0,help="update frequence")

aa("--mode", type=str,default="None")
aa('--alpha', '-a', type=float, default=0.12)

In [ ]:
train_data = np.load('./data/wiki_data_film/train_data.npy')
train_labels = np.load('./data/wiki_data_film/train_labels.npy')
test_data = np.load('./data/wiki_data_film/test_data.npy')
test_labels = np.load('./data/wiki_data_film/test_labels.npy')

all_gt = np.load('./data/wiki_data_film/gt.npy')

print("finish loading")
np.random.seed(1234)
tsize = 100000
tidx = np.random.choice(train_data.shape[0], tsize, replace=False)


xt_labels = train_labels[tidx]
xb_labels = train_labels
xb = train_data
xt = train_data[tidx]

imp_class = 1

imp_idx = xt_labels == imp_class 
test_imp_idx = test_labels == imp_class 
xb_imp = xb_labels == imp_class 
  

imp_test = np.nonzero(test_imp_idx)[0]
imp_xb = np.nonzero(xb_imp)[0]

dpr = []
xq_all = {}
gt_all = {}
all_skewness = [0.7]
l = [64]
for skewness in all_skewness:
  print("generating data for skwness {}".format(skewness))
  other_size = int(imp_test.shape[0]/skewness) - imp_test.shape[0]
  other_test = np.setdiff1d(np.arange(test_data.shape[0]),imp_test)
  other_idx = np.random.choice(other_test.shape[0], other_size , replace=False)
  other_test = other_test[other_idx]
  test_idx = np.concatenate([other_test,imp_test])

  xq = test_data[test_idx]
  print(xq.shape)
  gt = all_gt[test_idx,:]
  xq_all[skewness] = xq
  gt_all[skewness] = gt


opq_res = generate_opq(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)
opq_adp = generate_opq_adaptive(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)

hnsw = generate_hnsw([15],xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)


c_dir = "./wiki_film/model"
if(not os.path.exists(c_dir)):
  os.makedirs(c_dir)
args = parser.parse_args("--lr_schedule 0.1,0.1,0.05,0.01 --num_learn 100000 --database bigann --update_freq 20 --val_freq 10 --lambda_triplet 10 --margin_k 10 --lambda_uniform 0.1 --lambda_lid 0.05 --lambda_seperation 0.01   --dint 1024 --dout 64 --size_base 5000000 --batch_size 512 --epochs 60 --save_best_criterion opqAdaptive_64,rank=100".split())
log_res = generate_result(l,xt,xq_all,xb,imp_idx,xb_imp,gt_all,args=args,dir=c_dir,all_skewness=all_skewness)


In [ ]:
train_data = np.load('./data/wiki_data_soccer/train_data.npy')
train_labels = np.load('./data/wiki_data_soccer/train_labels.npy')
test_data = np.load('./data/wiki_data_soccer/test_data.npy')
test_labels = np.load('./data/wiki_data_soccer/test_labels.npy')

all_gt = np.load('./data/wiki_data_soccer/gt.npy')

print("finish loading")
np.random.seed(1234)
tsize = 100000
tidx = np.random.choice(train_data.shape[0], tsize, replace=False)


xt_labels = train_labels[tidx]
xb_labels = train_labels
xb = train_data
xt = train_data[tidx]

imp_class = 1

imp_idx = xt_labels == imp_class 
test_imp_idx = test_labels == imp_class 
xb_imp = xb_labels == imp_class 
  

imp_test = np.nonzero(test_imp_idx)[0]
imp_xb = np.nonzero(xb_imp)[0]

dpr = []
xq_all = {}
gt_all = {}
all_skewness = [0.7]
l = [64]
for skewness in all_skewness:
  print("generating data for skwness {}".format(skewness))
  other_size = int(imp_test.shape[0]/skewness) - imp_test.shape[0]
  other_test = np.setdiff1d(np.arange(test_data.shape[0]),imp_test)
  other_idx = np.random.choice(other_test.shape[0], other_size , replace=False)
  other_test = other_test[other_idx]
  test_idx = np.concatenate([other_test,imp_test])

  xq = test_data[test_idx]
  print(xq.shape)
  gt = all_gt[test_idx,:]
  xq_all[skewness] = xq
  gt_all[skewness] = gt


opq_res = generate_opq(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)
opq_adp = generate_opq_adaptive(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)

hnsw = generate_hnsw([15],xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)


c_dir = "./wiki_soccer/model"
if(not os.path.exists(c_dir)):
  os.makedirs(c_dir)
args = parser.parse_args("--lr_schedule 0.1,0.1,0.05,0.01 --num_learn 100000 --database bigann --update_freq 20 --val_freq 10 --lambda_triplet 10 --margin_k 10 --lambda_uniform 0.1 --lambda_lid 0.05 --lambda_seperation 0.01   --dint 1024 --dout 64 --size_base 5000000 --batch_size 512 --epochs 60 --save_best_criterion opqAdaptive_64,rank=100".split())
log_res = generate_result(l,xt,xq_all,xb,imp_idx,xb_imp,gt_all,args=args,dir=c_dir,all_skewness=all_skewness)


In [ ]:
train_data = np.load('./data/dpr_data/train_data.npy')
train_labels = np.load('./data/dpr_data/train_labels.npy')
test_data = np.load('./data/dpr_data/test_data.npy')
test_labels = np.load('./data/dpr_data/test_labels.npy')

all_gt = np.load('./data/dpr_data/gt.npy')

print("finish loading")
np.random.seed(1234)
tsize = 100000
tidx = np.random.choice(train_data.shape[0], tsize, replace=False)


xt_labels = train_labels[tidx]
xb_labels = train_labels
xb = train_data
xt = train_data[tidx]

imp_class = 2

imp_idx = xt_labels == imp_class 
test_imp_idx = test_labels == imp_class 
xb_imp = xb_labels == imp_class 
  
imp_test = np.nonzero(test_imp_idx)[0]
imp_xb = np.nonzero(xb_imp)[0]

dpr = []
xq_all = {}
gt_all = {}
all_skewness = [0.7]
l = [64]
for skewness in all_skewness:
  print("generating data for skwness {}".format(skewness))
  other_size = int(imp_test.shape[0]/skewness) - imp_test.shape[0]
  other_test = np.setdiff1d(np.arange(test_data.shape[0]),imp_test)
  other_idx = np.random.choice(other_test.shape[0], other_size , replace=False)
  other_test = other_test[other_idx]
  test_idx = np.concatenate([other_test,imp_test])

  xq = test_data[test_idx]
  print(xq.shape)
  gt = all_gt[test_idx,:]
  xq_all[skewness] = xq
  gt_all[skewness] = gt

opq_res = generate_opq(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)
opq_adp = generate_opq_adaptive(l,xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)

# hnsw = generate_hnsw([15],xt,xq_all,xb,imp_idx,gt_all,all_skewness=all_skewness)


c_dir = "./dpr/model"
if(not os.path.exists(c_dir)):
  os.makedirs(c_dir)
args = parser.parse_args("--num_learn 100000 --database bigann --lambda_triplet 100 --margin_k 10 --lambda_uniform 0.01 --lambda_lid 0.1 --lambda_seperation 0.1 --update_freq 20 --dint 1024 --dout 64 --size_base 500000 --batch_size 512 --epochs 120 --save_best_criterion opqAdaptive_64,rank=100".split())
log_res = generate_result(l,xt,xq_all,xb,imp_idx,xb_imp,gt_all,args=args,dir=c_dir,all_skewness=all_skewness)